## Uppgift 2 - Sport statistics

Date: 2021-11-10

Choose 2-4 sports and create suitable graphs / diagrams to visualize for example:
- medal distribution between countries in sports
- age distribution in sports
- Create more plots to visualize different aspects of the sport.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly_express as px
import plotly.graph_objects as go
from datetime import datetime
from load_data import ShowMeData

In [4]:
athlete_regions = pd.read_csv("data/athlete_regions.csv")
athlete_regions.head()

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal,region,notes,_merge
0,1,A Dijiang,M,24.0,180.0,80.0,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,NaN,China,NaN,both
1,2,A Lamusi,M,23.0,170.0,60.0,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,NaN,China,NaN,both
2,3,Gunnar Nielsen Aaby,M,24.0,NaN,NaN,Denmark,DEN,1920 Summer,1920,Summer,Antwerpen,Football,Football Men's Football,NaN,Denmark,NaN,both
3,4,Edgar Lindenau Aabye,M,34.0,NaN,NaN,Denmark/Sweden,DEN,1900 Summer,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold,Denmark,NaN,both
4,5,Christine Jacoba Aaftink,F,21.0,185.0,82.0,Netherlands,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,Speed Skating Women's 500 metres,NaN,Netherlands,NaN,both


In [6]:
athlete_regions = athlete_regions.iloc[:, :-2].reset_index(drop=True)
athlete_regions["Medal"] = athlete_regions["Medal"].fillna(0) 
athlete_regions.head()

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
0,1,A Dijiang,M,24.0,180.0,80.0,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,0
1,2,A Lamusi,M,23.0,170.0,60.0,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,0
2,3,Gunnar Nielsen Aaby,M,24.0,NaN,NaN,Denmark,DEN,1920 Summer,1920,Summer,Antwerpen,Football,Football Men's Football,0
3,4,Edgar Lindenau Aabye,M,34.0,NaN,NaN,Denmark/Sweden,DEN,1900 Summer,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold
4,5,Christine Jacoba Aaftink,F,21.0,185.0,82.0,Netherlands,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,Speed Skating Women's 500 metres,0


In [7]:
sports = athlete_regions['Sport'].unique().tolist()
sports

['Basketball',
 'Judo',
 'Football',
 'Tug-Of-War',
 'Speed Skating',
 'Cross Country Skiing',
 'Athletics',
 'Ice Hockey',
 'Swimming',
 'Badminton',
 'Sailing',
 'Biathlon',
 'Gymnastics',
 'Art Competitions',
 'Alpine Skiing',
 'Handball',
 'Weightlifting',
 'Wrestling',
 'Luge',
 'Water Polo',
 'Hockey',
 'Rowing',
 'Bobsleigh',
 'Fencing',
 'Equestrianism',
 'Shooting',
 'Boxing',
 'Taekwondo',
 'Cycling',
 'Diving',
 'Canoeing',
 'Tennis',
 'Modern Pentathlon',
 'Figure Skating',
 'Golf',
 'Softball',
 'Archery',
 'Volleyball',
 'Synchronized Swimming',
 'Table Tennis',
 'Nordic Combined',
 'Baseball',
 'Rhythmic Gymnastics',
 'Freestyle Skiing',
 'Rugby Sevens',
 'Trampolining',
 'Beach Volleyball',
 'Triathlon',
 'Ski Jumping',
 'Curling',
 'Snowboarding',
 'Rugby',
 'Short Track Speed Skating',
 'Skeleton',
 'Lacrosse',
 'Polo',
 'Cricket',
 'Racquets',
 'Motorboating',
 'Military Ski Patrol',
 'Croquet',
 'Jeu De Paume',
 'Roque',
 'Alpinism',
 'Basque Pelota',
 'Aeronautics'